### About the Notebook (Sentinel Export Country)

1.) This Notebbok is to export the country or state Sentinel Image which is created through the composition of ImageCollection. <br>
2.) Make sure that the country name is correct. <br>
3.) If the image is too big, it will split into pieces before being exported. The way it will be splitted depends on Google Earth Engine

In [ ]:
from datetime import datetime

In [ ]:
# Run only once
import ee

while True:
  try:
    ee.Authenticate()
    ee.Initialize()
    break
  except Exception as e:
    print()
    print('\033[1mAuthentication failed\033[0m')
    print(e)
    print()
    print('\033[1mPlease Try Again.\033[0m')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=5AumWTT_jKEmUR9j0Y6dK4tHAVhSTQ0-RuMH1Xij3Dk&tc=Q2gp8j6eI6IY5q5YtrkR_xQ8oopvIPBbAAfHEcjwKpo&cc=lobZX_FKo5Rm_LvQyJA7PKItfR78mxGxE_5YBYnJeew

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXm4YB8qaxMhUKqxUfPaAsnvWEGXpdcO6bXfXSynR8hY7LQn11luKx8

Successfully saved authorization token.


Get the max cloud probability.

In [ ]:
while True:
  try:
    MAX_CLOUD_PROBABILITY = int(input('Entert the max cloud probability: '))
    if MAX_CLOUD_PROBABILITY < 0 or MAX_CLOUD_PROBABILITY > 100:
      print('\033[1mInvalid cloud probability. It needs to be between 0 and 100.\033[0m')
      print()
    else:
      break
  except:
    print('\033[1mPlease enter a number.\033[0m')

Entert the max cloud probability: 30


In [ ]:
def get_sentinel_type():
  '''
  This function gets the sentinel type.

  Returns:
  - sentinel_type: str
  '''
  while True:
    try:
      # Ask for the Sentile Type
      sentinel_type = int(input('Enter the sentinel type: (Sentinel 1: 1, Sentinel 2: 2) - '))
      if sentinel_type == 1:
        return 'COPERNICUS/S2'
      elif sentinel_type == 2:
        return 'COPERNICUS/S2_SR'
      else:
        print('Please enter a valid sentinel type.')
    except:
      print('Please enter a number.')
      print()

In [ ]:
def get_area_type():
  '''
  This function gets the area type. Whether it is a State or a Country.

  Returns:
  - area_type: str
  '''
  while True:
    # ask for the area type
    area_type = str(input('Enter the area type: (State or Country) - ')).strip()
    if area_type == 'State':
      # If state, return state
      return 'State'
    elif area_type == 'Country':
      # If country, return country
      return 'Country'
    else:
      print('Please enter a valid area type. State or Country.')
      print()

In [ ]:
def is_valid_country(country, feature_collection_str):
  '''
  Checks if the country is valid.

  Parameters:
  - country_or_state (str): country
  - feature_collection_str (str): feature collection string

  Returns:
  bool: True if the country or state is valid, False otherwise
  '''
  # Loading the country's feature collection
  countries = ee.FeatureCollection(feature_collection_str)

  # Checking if the country is valid.
  # If the country will be valid, it will have 1 or 0 otherwise
  is_valid_country = countries.filter(
      ee.Filter.eq('country_na', country)).size().gt(0).getInfo()
  return is_valid_country

In [ ]:
def is_valid_state(state, feature_collection_str):
  '''
  Checks if the state is valid.

  Parameters:
  - state (str): state
  - feature_collection_str (str): feature collection string

  Returns:
  bool: True if the state is valid, False otherwise
  '''
  # Loading the state's feature collection
  states = ee.FeatureCollection(feature_collection_str)

  # Checking if the state is valid.
  # If the state is valid, it will be 1 or 0 otherwise
  is_valid_state = states.filter(
      ee.Filter.eq('ADM1_NAME', state)).size().gt(0).getInfo()
  return is_valid_state

In [ ]:
def get_area_name(area_type, feature_collection_str):
  '''
  This function gets the area name.

  Parameters:
  - area_type (str): area type
  - feature_collection_str (str): feature collection string

  Returns:
  - area_name: str
  - area_geometry: ee.Geometry
  '''
  while True:
    # Asking for the area name.
    area_name = str(input(f'Enter the name of the {area_type} - ')).strip()

    if area_type == 'Country':
      # If the area is country.
      if is_valid_country(area_name, feature_collection_str) == 1:
        # If the country name is valid.
        print(f'\033[1m{area_name} is valid.\033[0m')
        # getting the geometry of the country
        area_geometry = ee.FeatureCollection(feature_collection_str).filter(
            ee.Filter.eq('country_na', area_name)).geometry()
        return area_name, area_geometry
      else:
        # If the country name is not valid, continue asking for valid name.
        print(f'\033[1m{area_name} is not valid. Please try again.\033[0m')
        print()
    elif area_type == 'State':
      # If the area is state.
      if is_valid_state(area_name, feature_collection_str) == 1:
        # If the state name is valid.
        print(f'\033[1m{area_name} is valid.\033[0m')
        # Getting the geometry of the state
        area_geometry = ee.FeatureCollection(feature_collection_str).filter(
            ee.Filter.eq('ADM1_NAME', area_name)).geometry()
        return area_type, area_geometry
      else:
        # If the state name is not valid, continue asking for the valid name.
        print(f'\033[1m{area_name} is not valid. Please try again.\033[0m')
        print()

In [ ]:
def get_file_and_folder_name():
  '''
  This function gets the file name and folder name.

  Returns:
  - file_name: str
  - folder_name: str
  '''
  # Get the input for the file name and the file folder
  while True:
    file_name = str(input('Please enter the Image name - '))
    folder_name = str(input('Please enter the folder name - '))

    if file_name == '' or folder_name == '':
      print('\033[1mPlease enter a valid input.\033[0m')
      print()
    else:
      break
  print()
  print('\033[1mNote -\033[0m If there will be more than one AOI to export, a number will be prefixed to the file name.')
  print()
  return file_name, folder_name

In [ ]:
def is_valid_date_format(date_string):
  '''
  Checks if the date string is in the format YYYY-MM-DD

  Parameter:
  - date_string (str): date string in the format YYYY-MM-DD

  Returns:
  bool: True if the date string is in the format YYYY/MM/DD, False otherwise
  '''
  try:
    datetime.strptime(date_string, "%Y-%m-%d")
    return True
  except ValueError:
    return False

In [ ]:
def get_start_and_end_date():
  '''
  This function gets the start and end date.

  Returns:
  - start_date: str
  - end_date: str
  '''
  # Loop to get start date as the input.
  while True:
    start_date = str(input('Please enter the start date in following format (YYYY-MM-DD) - ')).strip()
    if is_valid_date_format(start_date):
      break
    else:
      print('\033[1mPlease enter a valid date.\033[0m')
      print()

  print()

  while True:
    end_date = str(input('Please enter the end date in following format (YYYY-MM-DD) - ')).strip()
    if is_valid_date_format(end_date):
      first_date = datetime.strptime(start_date, "%Y-%m-%d")
      second_date = datetime.strptime(end_date, "%Y-%m-%d")
      if first_date > second_date:
        print('\033[1mEnd date cannot be before start date. Please try again\033[0m')
        print()
      else:
        break
    else:
      print('\033[1mPlease enter a valid date.\033[0m')
      print()
  print()
  return start_date, end_date

In [ ]:
def mask_clouds(img):
  '''
  This function masks the clouds.

  Parameters:
  - img: ee.Image

  Returns:
  - ee.Image
  '''
  clouds = ee.Image(img.get('cloud_mask')).select('probability')
  is_not_cloud = clouds.lt(MAX_CLOUD_PROBABILITY)
  return img.updateMask(is_not_cloud)

In [ ]:
def mask_edges(s2_img):
  '''
  This function masks the edges.

  Parameters:
  - s2_img: ee.Image

  Returns:
  - ee.Image
  '''
  return s2_img.updateMask(
      s2_img.select('B8').mask().updateMask(s2_img.select('B9').mask())
  )

In [ ]:
def split_image_collection_by_month(collection):
  '''
  This function splits the image collection by month.

  Parameters:
  - collection: ee.ImageCollection

  Returns:
  - list of ee.ImageCollection
  '''
  return [collection.filter(ee.Filter.calendarRange(i, i, "month")) for i in range(1, 13)]

In [ ]:
def split_collection_img_mean(split_collection_list):
  '''
  This function does the mean on the image collection by month.

  Parameters:
  - split_collection_list: list of ee.ImageCollection

  Returns:
  - list of ee.Image
  '''
  median_list = []

  for img in split_collection_list:
    median_list.append(img.mean())
  return median_list

In [ ]:
def split_img_collection_gap_filled(split_collection_list):
  '''
  This function does the gap filling on the image collection by month.

  Parameters:
  - split_collection_list: list of ee.ImageCollection

  Returns:
  - list of ee.Image
  '''
  gap_filled_list = []

  for i in range(0, len(split_collection_list)):
    if i == 0:
      current_image = split_collection_list[i]
      next_image = split_collection_list[i+1]
      temp_image = ee.ImageCollection([current_image,
                                      next_image]).mean()
    elif i ==  len(split_collection_list) - 1:
      previous_image = split_collection_list[i-1]
      current_image = split_collection_list[i]
      temp_image = ee.ImageCollection([previous_image,
                                      current_image]).mean()
    else:
      previous_image = split_collection_list[i-1]
      current_image = split_collection_list[i]
      next_image = split_collection_list[i+1]
      temp_image = ee.ImageCollection([previous_image,
                                      current_image, next_image]).mean()
    gap_filled_list.append(temp_image)

  return gap_filled_list

In [ ]:
def to_double(image):
  '''
  This function converts the image to double.

  Parameters:
  - image: ee.Image

  Returns:
  - ee.Image
  '''
  return image.float()

In [ ]:
def get_bands_img(img_list, AOI):
  '''
  This function clears the bands and clips the image.

  Parameters:
  - img_list: ee.Image
  - AOI: ee.FeatureCollection

  Returns:
  - ee.Image
  '''
  band_list = ['B4', 'B3', 'B2']
  twelve_month_band_img = img_list.select(band_list).clip(AOI)
  return twelve_month_band_img

In [ ]:
def get_label_image(AOI):
  '''
  This function gets the label image.

  Parameters:
  - AOI: ee.FeatureCollection

  Returns:
  - ee.Image
  '''
  JB_mosaic_name = "projects/wri-datalab/TropicalTreeCover"
  jb_img_collection = ee.ImageCollection(JB_mosaic_name)
  jb_img = jb_img_collection.median().clip(AOI)
  return (jb_img.float())

In [ ]:
def add_labels(img, labels):
  '''
  This function adds the labels to the image.

  Parameters:
  - img: ee.Image
  - labels: ee.Image

  Returns:
  - ee.Image
  '''
  return img.addBands(labels.rename('target'))

### Exporting the Sentinel Image

After providing with the file name and the folder name, Sentinel images will be exported. The scale of the export will be 10 which is the original resolution of planet image.

You can look the current status of your export task at - [Export Tasks](https://code.earthengine.google.com/tasks)

In [ ]:
def export_image_collection_to_drive(img, folder_name, file_name, AOI):
  '''
  This function exports the image to the drive

  Parameters
  - img: The image needs to be exported
  - folder_name: The name of the folder in which the image needs to be exported
  - file_name: The name of the file
  - n: The number of the image
  - AOI: The area of interest
  '''
  description = f'{file_name}'
  img = img.clip(AOI)
  task = ee.batch.Export.image.toDrive(**{
      "image": img,
      "description": description,
      "folder": folder_name,
      "scale": 10, #This is the original scale for the Planet Image
      "maxPixels": 1e13,
      "region": AOI,
      'fileFormat': 'GeoTIFF'
  })

  task.start()
  print(f"Exporting image to drive - {description}")

In [ ]:
# Get the input for the file name and the file folder
while True:
  file_name = str(input('Please enter the file name - '))
  folder_name = str(input('Please enter the folder name - '))

  if file_name == '' or folder_name == '':
    print('\033[1mPlease enter a valid input.\033[0m')
    print()
  else:
    break
print()
print('\033[1mNote -\033[0m If there will be more than one AOI to export, a number will be prefixed to the file name.')
print()

Please enter the file name - andhra_pradesh_sentinel_pipeline_country
Please enter the folder name - Sentinel_Pipeline_Country

Note - If there will be more than one AOI to export, a number will be prefixed to the file name.



Get the area type. Is it a state or a country.

In [ ]:
area_type = get_area_type()

Enter the area type: (State or Country) - State


Get the name of the country and its boundary.

The geometry of the area types are depending on the FeatureCollection. The FeatureCollection are hardcoded below. They can be changed if needed.

In [ ]:
feature_collection_name_dic = {'State': 'FAO/GAUL/2015/level2',
                               'Country': 'USDOS/LSIB_SIMPLE/2017'}
feature_collection_str = feature_collection_name_dic[area_type]

In [ ]:
area_name, area_geometry = get_area_name(area_type, feature_collection_str)

Enter the name of the State - Andhra Pradesh
Andhra Pradesh is valid.


Get the Start and the End data

In [ ]:
start_date, end_date = get_start_and_end_date()

Please enter the start date in following format (YYYY-MM-DD) - 2016-01-01

Please enter the end date in following format (YYYY-MM-DD) - 2016-12-31



Get the Sentinel Type

In [ ]:
sentinel_type = get_sentinel_type()

Enter the sentinel type: (Sentinel 1: 1, Sentinel 2: 2) - 1


Export the image

**Run this bloack only if needed**

In [ ]:
# This is filtering criteria based on the start and end date
criteria = ee.Filter.And(ee.Filter.bounds(area_geometry),
                         ee.Filter.date(start_date, end_date))

s2sr = ee.ImageCollection(sentinel_type)
s2Clouds = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")
s2Sr = s2sr.filter(criteria).map(mask_edges)
s2Clouds = s2Clouds.filter(criteria)

# Applying the join on s2 and s2_clouds image collection for masking clouds
s2Sr_with_cloud_mask = ee.Join.saveFirst('cloud_mask').apply(**{
    'primary': s2Sr,
    'secondary': s2Clouds,
    'condition': ee.Filter.equals(**{
        'leftField': 'system:index',
        'rightField': 'system:index'
    })
})

# Masking the clouds
s2_cloud_masked = ee.ImageCollection(s2Sr_with_cloud_mask).map(mask_clouds)

# Split the image collection by months
monthly_list = split_image_collection_by_month(s2_cloud_masked)

# Get the mean image for each month
s2_mean_list = split_collection_img_mean(monthly_list)

# Gap fill the image from the previous and next month
s2_gap_filled_list = split_img_collection_gap_filled(s2_mean_list)

# Get the mean of all the images
s2_gap_filled_image = ee.ImageCollection.fromImages(s2_gap_filled_list).mean()

# Get the bands for the images
output_img = get_bands_img(s2_gap_filled_image, area_geometry)
output_img = output_img.float()

# Export the image to drive
export_image_collection_to_drive(output_img, folder_name, file_name, area_geometry)

Exporting image to drive - andhra_pradesh_sentinel_pipeline_country
